Importing

In [17]:
from rembg import remove 
import os
from PIL import Image 
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import shutil

Fix Rotation

In [18]:

def find_largest_id_card(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #Retrieve all non black pixels (discard the background)
    _, binary_mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0.0
    largest_contour = None

    for contour in contours:
        area = cv2.contourArea(contour)

        if area > max_area:
            max_area = area
            largest_contour = contour

    if largest_contour is not None:
        peri = cv2.arcLength(largest_contour, True)
        epsilon = 0.02 * peri
        rectangle_points = cv2.approxPolyDP(largest_contour, epsilon, True)
        return rectangle_points

    return None

def rotate_image(image, angle, scale=1.0):
    rows, cols = image.shape[:2]
    

    center = (cols / 2, rows / 2)

    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))
    return rotated_image

def rotate_points(points, angle, center):
    points_array = np.array(points, dtype=np.float32)
    points_array -= center
    theta = np.radians(angle)
    rotation_matrix = np.array([[np.cos(theta), np.sin(theta)],
                                [-np.sin(theta), np.cos(theta)]])
    rotated_points = np.dot(points_array, rotation_matrix.T)
    rotated_points += center
    #original format (list of tuples)
    rotated_points_list = rotated_points.reshape((-1, 2)).tolist()
    return rotated_points_list
    
def sort_points(pre_sort_points):
        sorted_points = sorted(pre_sort_points, key=lambda p: p[0])
        left_side = sorted_points[:2]
        right_side = sorted_points[2:]
        top_left = min(left_side, key=lambda p: p[1])
        bottom_left = max(left_side, key=lambda p: p[1])
        top_right = min(right_side, key=lambda p: p[1])
        bottom_right = max(right_side, key=lambda p: p[1])
        sorted_rectangle_points = [top_left, top_right, bottom_left, bottom_right]
        return sorted_rectangle_points
def fix_orientation(image,list_points,sorted_rectangle_points):
    # fontScale 
    fontScale = 1
    font = cv2.FONT_HERSHEY_SIMPLEX 
    color = (255, 0, 0) 
    thickness = 2
    if list_points is not None:
        x, y, w, h = cv2.boundingRect(list_points)
        #cropped_id_card = image[y:y + h, x:x + w]  
        rot_angle = 0
        while True:
            rotated_image = rotate_image(image,rot_angle)
            rotated_points = rotate_points(sorted_rectangle_points, rot_angle,(w // 2, h // 2))
            #cv2.drawContours(rotated_image, [np.array(rotated_points, dtype=np.int32)], 0, (0, 255, 0), 2)
            if abs(rotated_points[0][1] - rotated_points[1][1])>10 and abs(rotated_points[2][1] - rotated_points[3][1])>10:    
                if rotated_points[0][1] < rotated_points[1][1] and rotated_points[2][1] < rotated_points[3][1]:
                    rot_angle += 1
                else:
                    rot_angle-=1
                
            else:
                distance_01 = np.linalg.norm(np.array(rotated_points[0]) - np.array(rotated_points[1]))
                distance_02 = np.linalg.norm(np.array(rotated_points[0]) - np.array(rotated_points[2]))

                if distance_02 > distance_01:
                    rot_angle += 90 
                else:
                    rot_angle = rot_angle *1.04

                rotated_image = rotate_image(image, rot_angle)
                rotated_points = rotate_points(sorted_rectangle_points, rot_angle, (w // 2, h // 2))
                # for i, point in enumerate(rotated_points):
                #     rotated_image = cv2.putText(rotated_image, str(i), tuple(map(int, point)), font,  
                #                                 fontScale, color, thickness, cv2.LINE_AA) 
                return rotated_image

def run_rotation_fix(path):
    fontScale = 1
    font = cv2.FONT_HERSHEY_SIMPLEX 
    color = (255, 0, 0) 
    thickness = 2
    image = cv2.imread(path)
    image = remove(image) 
    # cv2.imshow('Rotated Image', image)
    list_points = find_largest_id_card(image)
    if list_points is None:
        #print('resized')
        image = cv2.resize(image, (1500, 1500))
        list_points = find_largest_id_card(image)
    #print(list_points)
    
    # for i, point in enumerate(list_points):
    #     print(point[0])
    #     image = cv2.putText(image, str(i),  (point[0]), font,  
    #             fontScale, color, thickness, cv2.LINE_AA) 

    list_points = list_points.copy().reshape((4, 2))
    sorted_rectangle_points = sort_points(list_points)
    # for i, point in enumerate(sorted_rectangle_points):
    #     print(point[0])
    #     image = cv2.putText(image, str(i),  tuple(map(int, point)), font,  
    #             fontScale, color, thickness, cv2.LINE_AA) 

    # cv2.imshow('Original Image', image)
    # cv2.waitKey(0)  # Wait for any key press



    rotated_image = fix_orientation(image,list_points,sorted_rectangle_points)
        
    base_name = os.path.splitext(os.path.basename(path))[0]
    output_path = os.path.join(base_name,'result0.jpg')
    cv2.imwrite(output_path, rotated_image)
    return(rotated_image)

CNN for upside down rotation.

In [19]:

def preprocess_image(image):
    image = image[:, :, :3]
    image = cv2.resize(image, (100, 100))
    image = np.expand_dims(image, axis=0)

    return image
def flip_check(image_path):
    model_path = 'flipped_detection_model.h5'
    model = load_model(model_path)
    input_image = preprocess_image(image_path)
    prediction = model.predict(input_image)
    if prediction[0][0] >= 0.5:
        return True
    else:
        return False


In [20]:
# import cv2
# image = cv2.imread(r'D:\Lines Segments_Abdalla Mohialdin_Cyshield\image1\result1.jpg')
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# #edge detection
# edges = cv2.Canny(blurred, 0, 150)

# cv2.imshow('z',edges)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Cropping

In [21]:


def detect_remove_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_area = 0.0
    largest_contour = None

    for contour in contours:
        area = cv2.contourArea(contour)

        if area > max_area:
            max_area = area
            largest_contour = contour

    if largest_contour is not None:
        peri = cv2.arcLength(largest_contour, True)
        epsilon = 0.02 * peri
        rectangle_points = cv2.approxPolyDP(largest_contour, epsilon, True)
        if len(rectangle_points) == 4:
            x, y, w, h = cv2.boundingRect(rectangle_points)
            cropped_id_card = image[y:y + h, x:x + w]
            flipped = flip_check(cropped_id_card)
            if flipped:
                cropped_id_card = rotate_image(cropped_id_card, 180)
            return cropped_id_card[20:-50, 10:-20]
    return None
def detect_id_card(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    #edge detection
    edges = cv2.Canny(blurred, 50, 150)
    return edges



def detect_and_highlight(image_canny,image_path):

    cropped_image = detect_remove_edges(image_canny)
    if cropped_image is None:

        image_canny = cv2.resize(image_canny, (1500, 1500))
        cropped_image = detect_remove_edges(image_canny)
    #print(cropped_image)
    
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    #print(base_name)
    output_path = os.path.join(base_name,'result1.jpg')
    cv2.imwrite( output_path, cropped_image)
    #print(output_path)

    edges = detect_id_card(cropped_image)

    output_path2 = os.path.join(base_name,'result2.jpg')
    cv2.imwrite(output_path2, edges)
    return edges



pre-processing for info extraction.

In [22]:


def dilating_image(image):

    binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    #remove noise
    median_filtered_image = cv2.medianBlur(binary_image, 3)

    edges = edge_detection(median_filtered_image)
    dilated_image = custom_dilation(edges)

    return dilated_image


def edge_detection(image):
    kernel = np.ones((3, 3), np.uint8)
    morph_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    edges = cv2.Canny(morph_image, 50, 150)

    return edges

def custom_dilation(image):
    #dilation_kernel = np.array([[0, 1, 0],
                                # [1, 1, 1],
                                # [0, 1, 0]], dtype=np.uint8)
    dilation_kernel = np.ones((1, 11), np.uint8)
    dilated_image = cv2.dilate(image, dilation_kernel, iterations=4)

    return dilated_image


def pre_process_before_detection(output_image,image_path):
    preprocessed_image = dilating_image(output_image)
    # cv2.imshow('original', cv2.imread(image_path))
    # cv2.imshow('preprocessed', preprocessed_image)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_path = os.path.join(base_name,'result3.jpg')
    cv2.imwrite(output_path,preprocessed_image)
    return preprocessed_image



Extracting important information.

In [23]:

def calculate_intersection(rect1, rect2):
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2

    x_intersection = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
    y_intersection = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))

    intersection_area = x_intersection * y_intersection
    area_rect1 = w1 * h1
    area_rect2 = w2 * h2
    if area_rect1 < area_rect2:
        intersection_percent = intersection_area / area_rect1
    else:
        intersection_percent = intersection_area / area_rect2
    #iou = intersection_area / float(area_rect1 + area_rect2 - intersection_area)
    return intersection_percent

def merge_overlapping_rectangles(rectangles):
    if not rectangles:
        return []

    merged_rectangles = [rectangles[0]]

    for current_rect in rectangles[1:]:
        #check occlusion for the new merged rectangle
        intersection = calculate_intersection(current_rect, merged_rectangles[-1])

        if intersection >= 0.4: 
            merged_rectangles[-1] = (
                min(merged_rectangles[-1][0], current_rect[0]),
                min(merged_rectangles[-1][1], current_rect[1]),
                max(merged_rectangles[-1][0] + merged_rectangles[-1][2], current_rect[0] + current_rect[2]) - min(merged_rectangles[-1][0], current_rect[0]),
                max(merged_rectangles[-1][1] + merged_rectangles[-1][3], current_rect[1] + current_rect[3]) - min(merged_rectangles[-1][1], current_rect[1])
            )
        else:
            merged_rectangles.append(current_rect)

    return merged_rectangles

def highlight_rectangles(image,main_image):

    min_rectangle_area=4000
    _, binary_mask = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    rectangles = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h > min_rectangle_area:
            rectangles.append((x, y, w, h))

    merged_rectangles = merge_overlapping_rectangles(rectangles)


    highlighted_image = main_image.copy()

    for rect in merged_rectangles:
        x, y, w, h = rect
        cv2.rectangle(highlighted_image, (x, y), (x + w, y + h), (0, 255, 0), 2)


    return highlighted_image, merged_rectangles

def detection_and_segmentation(image,image_path):
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    main_image_path = os.path.join(base_name, 'result1.jpg')
    main_image = cv2.imread(main_image_path)
    highlighted_image, merged_rectangles = highlight_rectangles(image,main_image)

    base_name = os.path.splitext(os.path.basename(image_path))[0]    
    for i, rect in enumerate(merged_rectangles):
        x, y, w, h = rect
        segment = main_image[y:y + h, x:x + w]
        
        output_path_segment = os.path.join(base_name, f'segment{i}.jpg')
        cv2.imwrite(output_path_segment, segment)

    output_path = os.path.join(base_name,'result4.jpg')
    cv2.imwrite(output_path, highlighted_image)


Running all code segments.

In [24]:

def facade_pattern(image_path):
    output_image = run_rotation_fix(image_path)
    output_image = detect_and_highlight(output_image,image_path)
    output_image = pre_process_before_detection(output_image,image_path)
    detection_and_segmentation(output_image,image_path)
    

def create_directory(image_path, base_directory):
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    directory_path = os.path.join(base_directory, base_name)
    os.makedirs(directory_path, exist_ok=True)
    facade_pattern(image_path)

    shutil.copy(image_path, os.path.join(directory_path, os.path.basename(image_path)))



def main():
    base_directory = os.getcwd()
    testing_directory = os.path.join(base_directory, 'testing')
    for filename in os.listdir(testing_directory):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(testing_directory, filename)
            #print(image_path)
            create_directory(image_path, base_directory)

if __name__ == "__main__":
    main()


1/1 [==============================] - 0s 64ms/step
